In [1]:
import CUBRIDdb
import pandas as pd
from datetime import datetime, timedelta
import sys

In [2]:
sys.stdout = open('/app/script/water_ans/ans_files/error_log.txt','a')

In [3]:
#today = datetime.now()
today = datetime(2021, 1, 6)
today = today.strftime('%Y-%m-%d')#ex)2019-03-14

In [4]:
conn = CUBRIDdb.connect('CUBRID:xxx.xx.xx.xxx:30000:dgsmwtanalysis:::','xxxxxx','xxxxxxx')
conn.set_autocommit(False)
cur = conn.cursor()
select_query= """\
    SELECT
        cstmr_no,
        mrnr_no,
        SUM(smrt_mtinsp_hour_usgqty) AS usgqty,
        MAX(smrt_mtinsp_indctpnt) AS max_indctpnt, 
        MIN(smrt_mtinsp_indctpnt) AS min_indctpnt
    FROM
        tb_smrt_mtinsp_info
    WHERE mrnr_no = 'A00'
    AND creat_stle = 'N'
    AND mtinsp_dt >= ADD_MONTHS(DATE '{today}', -3)
    GROUP BY cstmr_no, mrnr_no
    HAVING usgqty = 0
""".format(today=today)
select_query = select_query.replace("\n\t"," ")
select_query = select_query.replace("\n"," ")
cur.execute(select_query)

rows = cur.fetchall()
df = pd.DataFrame(rows)
df.columns=['cstmr_no', 'mrnr_no', 'usgqty','max_indctpnt','min_indctpnt']

In [5]:
df['cal'] = df['max_indctpnt']-df['min_indctpnt']

In [6]:
df_cal = df[df['cal'] == 0]

In [7]:
try:
    for index, row in df_cal.iterrows():
        cstmr_no = row["cstmr_no"]
        mrnr_no = row["mrnr_no"]
        insert_query = """\
            INSERT INTO TB_UNINHBHOUS_ANALS_TEST
            (ANALS_STDR_DAY,
            CSTMR_NO,
            MRNR_NO,
            FRST_REGIST_DT) 
            VALUES
            (DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
            '{cstmr_no}',
            '{mrnr_no}',
            CURRENT_TIMESTAMP
            )""".format(cstmr_no=cstmr_no, mrnr_no=mrnr_no)

        insert_query = insert_query.replace("\n\t"," ")
        insert_query = insert_query.replace("\n"," ")
        cur.execute(insert_query)
except Exception as e:
    print("빈집추정에러 ::: " + str(datetime.now()))
    print(e)
    conn.rollback()
else:
    conn.commit()